# Section 2) Explain your data set, how it is composed, and how you collected it (30%)

## data explanation
#### Q1: Age
#### Q2: What category do you fall under? (undergraduate or Graduate)
#### Q9: Are you employed?
#### Q10: What is your average monthly income, including money from outside sources such as stipend scholarships, or allowance from your parents?
#### Q17: Have you ever invested?
#### Q18: If you have invested, what have you invested on (select more than one)?
#### Q20: What is your average monthly spending (not including rent/utilities)?
#### Q21: What do you spend the most money on?
#### Q27: What do you think is the minimum amount needed to make an investment?
#### Q28: What do you usually spend your credit card cash back on?
#### Q29: On a scale of 1 to 5, 1 being not at all and 5 being definitely, how likely would you be to invest your cash back bonus? 
#### Q30: What is your area of study (you may select more than one option)?
#### Q31: "You are given the option to choose Credit Card A or Credit Card B each with the following cash back options:
#### Credit card A:-Redeem as cash-Apply to statement-Redeem as gift card ($20 minimum)
#### Credit Card B:-Redeem as cash-Apply to statement-Invest in a stock (no minimum)
#### Which credit card option would you be likely to choose?"
#### Q32: Would you rather invest your own money or money received from a cash back bonus?
#### Q33: On a scale of 1 to 5, 1 being not at all and 5 being definitely, how likely would you be to invest your own money? 
#### Q43: What characteristics would you/do you look for when considering an investment?
#### Q44: What is more important to you when making an investment? - Selected Choice(Higher return, saving, social responsibility) 


In [26]:
# The data is collected from survery that distributed in school

# Section 3) Load of the data set (10%)

In [28]:
import pandas as pd
import re
import numpy as np

# preprocess the data
# Extracting Missing Count and Unique Count by Column

data = pd.read_csv('Shi_and_Song_week10_data.csv')

# Section 4) Display data (50%)

In [29]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   StartDate              48 non-null     object
 1   EndDate                48 non-null     object
 2   Status                 48 non-null     object
 3   Duration (in seconds)  48 non-null     object
 4   Finished               48 non-null     object
 5   RecordedDate           48 non-null     object
 6   DistributionChannel    48 non-null     object
 7   Q36                    47 non-null     object
 8   Q1                     44 non-null     object
 9   Q2                     44 non-null     object
 10  Q30                    44 non-null     object
 11  Q9                     45 non-null     object
 12  Q10                    43 non-null     object
 13  Q20                    43 non-null     object
 14  Q21                    44 non-null     object
 15  Q17                    39

In [23]:
# drop unnecessary data
data.drop([0,1],inplace=True)
data.drop(columns=['EndDate','Duration (in seconds)','Status','RecordedDate',
                   'RecipientLastName','DistributionChannel','Q36'],axis=1,inplace=True)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 2 to 47
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   StartDate           46 non-null     object
 1   IPAddress           45 non-null     object
 2   Progress            46 non-null     object
 3   Finished            46 non-null     object
 4   ResponseId          46 non-null     object
 5   RecipientFirstName  0 non-null      object
 6   RecipientEmail      0 non-null      object
 7   ExternalReference   0 non-null      object
 8   LocationLatitude    40 non-null     object
 9   LocationLongitude   40 non-null     object
 10  UserLanguage        46 non-null     object
 11  Q1                  42 non-null     object
 12  Q2                  42 non-null     object
 13  Q30                 42 non-null     object
 14  Q9                  43 non-null     object
 15  Q10                 41 non-null     object
 16  Q20                 41 non-n

In [4]:
# fill in some typo data
data.iloc[7,data.columns.get_loc("Q20")]=175
data.iloc[33,data.columns.get_loc("Q20")]=1750
data=data[data['Finished']=='True']

In [5]:
# check out unique and missing data
unique_count = []
for x in data.columns:
    unique_count.append([x,len(data[x].unique()),data[x].isnull().sum()])
con=pd.DataFrame(unique_count, columns=["Column","Unique","Missing"]).set_index("Column").T
print(con)

Column   StartDate  Finished  Q1  Q2  Q30  Q9  Q10  Q20  Q21  Q17  ...  Q31  \
Unique          40         1   9   3    7   2   27   23    6    8  ...    2   
Missing          0         0   0   0    0   0    1    1    0    4  ...    0   

Column   Q32  Q33_1  Q29_1  Q43  Q44  Q44_4_TEXT  Q45_1  Q45_2  Q45_3  
Unique     4      6      6   22    4           2      4      4      4  
Missing    2      2      2   19   14          39     20     20     20  

[2 rows x 25 columns]


In [6]:
# clear data and manage data type
data['Average_income'] = data['Q10'].apply(lambda x: re.sub("[\s+\.\!\/_,$%^*(+\"\')+\]]+|[+——([)?【】“”！，\-。？、"
                                                            "~@#￥%……&*（）]+", '', str(x)))
data['Average_spend'] = data['Q20'].apply(lambda x: re.sub("[\s+\.\!\/_,$%^*(+\"\')+\]]+|[+——([)?【】“”！，\-。"
                                                           "？、~@#￥%……&*（）]+", '', str(x)))
data['Average_income'] = pd.to_numeric(data['Average_income'], errors='coerce')
data['Average_spend'] = pd.to_numeric(data['Average_spend'], errors='coerce')
data.loc[data['Average_spend'] ==0,'Average_spend']=np.nan
data.loc[data['Average_income'] ==0,'Average_income']=np.nan

# find missing value
miss_income_col = data.loc[data['Average_income'].isnull()]['Average_spend'].unique()
print(miss_income_col)

[650. 300.]


In [7]:
# fill missing value in income
def fill_na_with_mean1(ds, value):
  fill_value = ds.loc[ds['Average_spend'] ==value]['Average_income'].mean()
  condit = ((ds['Average_spend'] == value) & (ds['Average_income'].isnull()))
  ds.loc[condit, 'Average_income'] = ds.loc[condit, 'Average_income'].fillna(fill_value)
for a in miss_income_col:
  fill_na_with_mean1(data, a)

miss_spend_col = data.loc[data['Average_spend'].isnull()]['Average_income'].unique()
print(miss_spend_col)

[1000.]


In [10]:
# fill missing value in spend
def fill_na_with_mean2(ds, value):
  fill_value = ds.loc[ds['Average_income'] ==value]['Average_spend'].mean()
  condit = ((ds['Average_income'] == value) & (ds['Average_spend'].isnull()))
  ds.loc[condit, 'Average_spend'] = ds.loc[condit, 'Average_spend'].fillna(fill_value)
for a in miss_spend_col:
  fill_na_with_mean2(data, a)

In [17]:
# check null value in spend and income
data.sort_values(by="Average_spend", ascending=True)
data['Average_income']= data['Average_income'].interpolate()
data['Average_spend'].isna().sum()

0

In [18]:
data

,StartDate,Finished,Q1,Q2,Q30,Q9,Q10,Q20,Q21,Q17,...,Q33_1,Q29_1,Q43,Q44,Q44_4_TEXT,Q45_1,Q45_2,Q45_3,Average_income,Average_spend
2,2019-11-11 08:18:20,True,23,Graduate (Masters),"Business (finance, management, accounting, etc.)",Yes,"$1,300",$450,Other,4,...,4,2,NaN,NaN,NaN,NaN,NaN,NaN,1300.0,450.0
3,2019-11-11 08:31:26,True,22,Graduate (Masters),"Humanities (philosophy, history, english, educ...",Yes,$1000,$500,Groceries/day-to-day needs,1,...,3,3,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,500.0
5,2019-11-11 16:07:59,True,22,Graduate (Masters),"Business (finance, management, accounting, etc.)",Yes,1200,400,Groceries/day-to-day needs,4,...,5,5,NaN,NaN,NaN,NaN,NaN,NaN,1200.0,400.0
6,2019-11-12 20:25:07,True,24,Graduate (Masters),"Business (finance, management, accounting, etc.)",No,0,650,Groceries/day-to-day needs,1,...,4,5,NaN,NaN,NaN,NaN,NaN,NaN,950.0,650.0
7,2019-11-13 15:13:30,True,23,Graduate (Masters),"Business (finance, management, accounting, etc.)",Yes,700,300,Other,4,...,5,5,NaN,NaN,NaN,NaN,NaN,NaN,700.0,300.0
8,2019-11-13 15:16:31,True,22,Graduate (Masters),"Business (finance, management, accounting, etc.)",Yes,1300,900,"Eating out (restaurants, coffee, etc.)",5,...,3,4,NaN,NaN,NaN,NaN,NaN,NaN,1300.0,900.0
9,2019-11-13 15:17:35,True,22,Graduate (Masters),"Business (finance, management, accounting, etc.)",No,800,175,Groceries/day-to-day needs,3,...,2,3,NaN,NaN,NaN,NaN,NaN,NaN,800.0,175.0
10,2019-11-13 15:18:45,True,22,Graduate (Masters),"Business (finance, management, accounting, etc.)",No,880,175,"Eating out (restaurants, coffee, etc.)",4,...,4,5,NaN,NaN,NaN,NaN,NaN,NaN,880.0,175.0
11,2019-11-13 15:22:26,True,23,Graduate (Masters),"Business (finance, management, accounting, etc.)",No,800,500,Travel,4,...,4,5,NaN,NaN,NaN,NaN,NaN,NaN,800.0,500.0
12,2019-11-13 20:46:36,True,24,Graduate (Masters),"Business (finance, management, accounting, etc.)",Yes,1000,600,Groceries/day-to-day needs,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,600.0


In [19]:
# prepare selected feature
data = data.rename(columns={'Q1':'Age', 'Q9':'Employed', 'Q28':'Cashback_spending', 'Q21':'Most_expense', 'Q31':'Option',
                           'Q2':'Education_level'})
data_ml=data[['Age', 'Employed', 'Cashback_spending', 'Most_expense', 'Option', 'Average_income', 'Education_level']]

In [20]:
data_ml.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 2 to 47
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                40 non-null     object 
 1   Employed           40 non-null     object 
 2   Cashback_spending  38 non-null     object 
 3   Most_expense       40 non-null     object 
 4   Option             40 non-null     object 
 5   Average_income     40 non-null     float64
 6   Education_level    40 non-null     object 
dtypes: float64(1), object(6)
memory usage: 2.5+ KB


In [21]:
data_ml

,Age,Employed,Cashback_spending,Most_expense,Option,Average_income,Education_level
2,23,Yes,Apply to statement balance,Other,Option A,1300.0,Graduate (Masters)
3,22,Yes,Apply to statement balance,Groceries/day-to-day needs,Option A,1000.0,Graduate (Masters)
5,22,Yes,Redeem as cash,Groceries/day-to-day needs,Option B,1200.0,Graduate (Masters)
6,24,No,Redeem as cash,Groceries/day-to-day needs,Option B,950.0,Graduate (Masters)
7,23,Yes,NaN,Other,Option B,700.0,Graduate (Masters)
8,22,Yes,Apply to statement balance,"Eating out (restaurants, coffee, etc.)",Option B,1300.0,Graduate (Masters)
9,22,No,Apply to statement balance,Groceries/day-to-day needs,Option A,800.0,Graduate (Masters)
10,22,No,Apply to statement balance,"Eating out (restaurants, coffee, etc.)",Option B,880.0,Graduate (Masters)
11,23,No,Apply to statement balance,Travel,Option B,800.0,Graduate (Masters)
12,24,Yes,Apply to statement balance,Groceries/day-to-day needs,Option B,1000.0,Graduate (Masters)
